In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy import stats
import json
from grade_rank_calculation import calculate_grade_rank
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly.offline as offline
from geopy.geocoders import Nominatim

In [2]:
# (1) read data

df = pd.read_pickle('CuratedWithRatings_OpenBetaAug2020_RytherAnderson.pkl.zip', compression='zip')
df = df.where(pd.notnull(df), None)

# splite user_ratings into two columns for ease of use
df['users'] = df.apply(lambda row: [r[0] for r in row['corrected_users_ratings']], axis = 1)
df['ratings'] = df.apply(lambda row: [r[1] for r in row['corrected_users_ratings']], axis = 1)

In [3]:
# (2) new variables, list comprehension is much faster than apply

df['mean_rating'] = [np.mean(r) for r in df.ratings.values]
df['median_rating'] = [np.median(r) for r in df.ratings.values]
df['mode_rating'] = [stats.mode(r)[0][0] for r in df.ratings.values]
df['num_votes'] = [len(r) for r in df.ratings.values]

df['RQI_mean'] = df['mean_rating'] * (1.0 - (1.0/df['num_votes']))
df['RQI_median'] = df['median_rating'] * (1.0 - (1.0/df['num_votes']))

In [4]:
# (3) make subsets and calculate new categorization variables
# There are not many boulders in the dataset with user ratings (234), so boulders are not used in this analysis.
# Mixed routes are lumped into a new category with trad (trad_mixed = routes that take gear).
# Seven sport climbs are only given V grades, these are removed.

df = df[df['type_string'] != 'boulder']
df = df[df['YDS_rank'].notnull()]
df.loc[df['type_string'] == 'mixed', 'type_string'] = 'trad'
df['YDS_rank'] = df['YDS_rank'].astype(int)
df['VCID'] = df['YDS_rank'].astype(str) + df['type_string'] # vote count ID

sport = df[df['type_string'] == 'sport'].copy()
trad_mixed = df[df['type_string'] == 'trad'].copy()

total = len(df.index)
print(total, 'total climbs after removing boulders')

96722 total climbs after removing boulders


In [5]:
# (3) adjusted RQI, meant to account for harder routes seeing fewer ascents.
# Essentially, votes are weighted more for harder routes. The weights are calculated from the distribution of mean vote counts.
# Weights are calculated separately for sport and trad, since these types have different mean vote count distributions.

SC = sport.groupby('VCID', as_index=False).agg({'num_votes': ['sum', 'count']})
TC = trad_mixed.groupby('VCID', as_index=False).agg({'num_votes': ['sum', 'count']})

SC['VPR'] = SC[('num_votes','sum')]/SC[('num_votes','count')]
TC['VPR'] = TC[('num_votes','sum')]/TC[('num_votes','count')]

SC['VCAF'] = 1.0/(SC['VPR']/max(SC['VPR'])) # VCAF = vote count adjustment factor
TC['VCAF'] = 1.0/(TC['VPR']/max(TC['VPR']))

SC = SC.drop('num_votes', 1)
TC = TC.drop('num_votes', 1)

VC = pd.concat([SC, TC])
VC.columns = [''.join(col) if type(col) is tuple else col for col in SC.columns.values]
df = pd.merge(df, VC, on='VCID')

df['adjusted_num_votes'] = df['num_votes'] * df['VCAF']
df['ARQI_mean'] = df['mean_rating'] * (1.0 - (1.0/df['adjusted_num_votes']))
df['ARQI_median'] = df['median_rating'] * (1.0 - (1.0/df['adjusted_num_votes']))

/Users/Ryther/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:3887: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [6]:
# (4) assessing different metrics for finding "the best routes"

df_quality = df[['route_name', 'route_ID', 'type_string', 'num_votes', 
                 'adjusted_num_votes', 'mean_rating', 'median_rating', 'mode_rating',
                 'RQI_mean', 'RQI_median', 'ARQI_mean', 'ARQI_median', 'nopm_YDS', 'YDS_rank']].copy()

def find_best_routes(df, metrics, num=3, route_type='all', grade_range='all'):
    
    if route_type != 'all':
        df = df[df['type_string'] == route_type].copy()
    
    if grade_range != 'all':
        lo,hi = grade_range
        lo_rank = calculate_grade_rank(lo)
        hi_rank = calculate_grade_rank(hi)                
        df = df[(lo_rank <= df['YDS_rank']) & (df['YDS_rank'] <= hi_rank)].copy()
    
    rankings = []
    for metric in metrics:
        rank_name =  '_'.join([metric, 'rank'])
        df[rank_name] = df[metric].rank(method='dense', ascending=False)
        rankings.append(rank_name)
    
    df['total_rank'] = 0
    for rank in rankings:
        df['total_rank'] += df[rank]
    df['average_rank'] = df['total_rank']/len(rankings)
    
    df = df.sort_values(by=['average_rank'])
    routes = df.head(num)
    
    return routes
    
find_best_routes(df_quality, ['ARQI_median'], num=5, grade_range=('5.12a','5.13a'), route_type='sport')

,route_name,route_ID,type_string,num_votes,adjusted_num_votes,mean_rating,median_rating,mode_rating,RQI_mean,RQI_median,ARQI_mean,ARQI_median,nopm_YDS,YDS_rank,ARQI_median_rank,total_rank,average_rank
69661,Ten-Digit Dialing,105748421,sport,333,1180.067103,3.633634,4.0,4.0,3.622722,3.987988,3.630554,3.996610,5.12c,124,1.0,1.0,1.0
20119,Fission aka Ken T'ank,105758647,sport,221,1035.858601,3.647059,4.0,4.0,3.630556,3.981900,3.643538,3.996138,5.12b/c,123,2.0,2.0,2.0
74097,Lucid Dreaming,105755821,sport,153,1003.135931,3.712418,4.0,4.0,3.688154,3.973856,3.708717,3.996013,5.12c/d,125,3.0,3.0,3.0
20105,The Great Escape,105748433,sport,212,993.674314,3.683962,4.0,4.0,3.666585,3.981132,3.680255,3.995975,5.12b/c,123,4.0,4.0,4.0
17207,Ro Shampo,105860759,sport,414,839.382034,3.685990,4.0,4.0,3.677087,3.990338,3.681599,3.995235,5.12a,120,5.0,5.0,5.0


In [8]:
df_save = df[['route_name', 'route_ID', 'type_string', 'sector_ID', 'parent_sector', 'parent_loc', 'num_votes', 
              'adjusted_num_votes', 'mean_rating', 'median_rating', 'mode_rating', 'RQI_mean', 
              'RQI_median', 'ARQI_mean', 'ARQI_median', 'nopm_YDS', 'YDS_rank', 'safety']].copy()

df_save.to_pickle('RouteQualityData.pkl.zip', compression='zip')

,route_name,parent_sector,route_ID,sector_ID,type_string,fa,YDS,Vermin,nopm_YDS,nopm_Vermin,...,mode_rating,num_votes,RQI_mean,RQI_median,VCID,VPR,VCAF,adjusted_num_votes,ARQI_mean,ARQI_median
0,Stairway to Heaven,Drive In Wall,106956280,106947227,trad,unknown,5.7,None,5.7,None,...,3.0,10,2.340000,2.7,73trad,21.682407,1.198552,11.985516,2.383072,2.749698
1,Dirty Lid,Ice Cave Wall,106580141,106087932,trad,unknown,5.7,None,5.7,None,...,1.0,2,0.500000,0.5,73trad,21.682407,1.198552,2.397103,0.582830,0.582830
2,Herniated Disk,Backbone East,106957884,106956366,trad,unknown,5.7,None,5.7,None,...,3.0,3,1.777778,2.0,73trad,21.682407,1.198552,3.595655,1.925031,2.165660
3,Cole's Calamity,Backbone East,106957920,106956366,trad,unknown,5.7,None,5.7,None,...,1.0,1,0.000000,0.0,73trad,21.682407,1.198552,1.198552,0.165660,0.165660
4,Ring Trilogy,Palisades-Dows Preserve,113690469,113690461,trad,unknown,5.7,None,5.7,None,...,3.0,1,0.000000,0.0,73trad,21.682407,1.198552,1.198552,0.496979,0.496979
